# 🧹 Data Transformation - Cleaning and Enriching Your Data

Welcome to the fifth tutorial in our **Data Ingestion Pipeline** series! In this hands-on notebook, you'll learn how to transform raw, messy data into clean, standardized, and enriched data ready for analysis.

## 🎯 Learning Objectives

By the end of this tutorial, you will:
- ✅ Clean messy data and handle missing values
- ✅ Standardize data formats and structures
- ✅ Enrich data with calculated fields and metadata
- ✅ Normalize and deduplicate datasets
- ✅ Build transformation pipelines
- ✅ Monitor transformation quality and performance

---

## 🛠️ Setup and Imports

Let's start by importing the libraries we'll need for data transformation:

In [ ]:
# Essential imports for data transformation
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple, Union
import warnings
warnings.filterwarnings('ignore')

# For visualizations
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("husl")

# For data transformation
from dataclasses import dataclass
import json
from collections import Counter
import hashlib

print("📦 All libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"⏰ Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🎭 The Data Transformation Journey

Data transformation is like **renovating a house** - you take something rough and make it beautiful and functional:

### 🏠 **Before Transformation (Raw Data):**
- 🧱 **Messy Structure** - Inconsistent formats, missing values
- 🎨 **No Standards** - Different naming conventions, mixed cases
- 🔧 **Basic Functionality** - Data exists but hard to use
- 📏 **No Measurements** - Missing calculated fields and metrics

### ✨ **After Transformation (Clean Data):**
- 🏛️ **Beautiful Structure** - Consistent, standardized format
- 🎯 **Clear Standards** - Uniform naming, proper data types
- 🚀 **Enhanced Functionality** - Ready for analysis and ML
- 📊 **Rich Information** - Enriched with insights and metadata

In [ ]:
# Create sample messy data to demonstrate transformations
print("🎭 Creating Sample Messy Data")
print("=" * 30)

# Simulate real-world messy e-commerce data
messy_orders = {
    'order_id': [
        'ORD-2024-001',    # ✅ Good format
        'ord-2024-002',    # 🔧 Wrong case
        'ORD_2024_003',    # 🔧 Wrong separator
        'ORDER-2024-004',  # 🔧 Too long prefix
        'ORD-2024-001',    # ❌ Duplicate
        '2024-005',        # 🔧 Missing prefix
        'ORD-2024-007',    # ✅ Good format
        'ORD-24-008'       # 🔧 Wrong year format
    ],
    'customer_name': [
        'John Doe',           # ✅ Good format
        'jane smith',         # 🔧 Wrong case
        'BOB WILSON',         # 🔧 All caps
        'Alice  Johnson',     # 🔧 Extra spaces
        'charlie brown jr.',  # 🔧 Mixed case
        'Dr. Sarah Connor',   # 🔧 Title included
        'Mike O\'Brien',      # 🔧 Apostrophe
        'Emma-Rose Davis'     # 🔧 Hyphenated
    ],
    'customer_email': [
        'john@example.com',      # ✅ Good format
        'JANE@COMPANY.COM',      # 🔧 Wrong case
        'bob@email.co.uk',       # ✅ Good format
        'alice@domain',          # ❌ Incomplete
        'charlie.brown@test.org', # ✅ Good format
        'sarah.connor@skynet.mil', # ✅ Good format
        'mike@',                 # ❌ Incomplete
        'emma.davis@company.com' # ✅ Good format
    ],
    'product_name': [
        'iPhone 15',          # ✅ Good format
        'macbook pro',        # 🔧 Wrong case
        'AIRPODS PRO',        # 🔧 All caps
        'iPad  Air',          # 🔧 Extra spaces
        'apple watch series 9', # 🔧 Wrong case
        'Nintendo Switch OLED', # 🔧 Mixed case
        'samsung galaxy s24',   # 🔧 Wrong case
        'Sony WH-1000XM5'     # ✅ Good format
    ],
    'quantity': [
        '1',      # 🔧 String instead of int
        2,        # ✅ Good format
        '3.0',    # 🔧 Float string
        1,        # ✅ Good format
        '2',      # 🔧 String
        1,        # ✅ Good format
        '1',      # 🔧 String
        2         # ✅ Good format
    ],
    'price': [
        '$999.99',     # 🔧 Currency symbol
        '1999.99',     # 🔧 String
        249.99,        # ✅ Good format
        '599.99 USD',  # 🔧 Currency code
        399.99,        # ✅ Good format
        '$299.99',     # 🔧 Currency symbol
        '899.99',      # 🔧 String
        549.99         # ✅ Good format
    ],
    'order_date': [
        '2024-01-15',      # ✅ Good format
        '01/16/2024',      # 🔧 US format
        '17-01-2024',      # 🔧 DD-MM-YYYY
        '2024/01/18',      # 🔧 Slash separator
        '19 Jan 2024',     # 🔧 Text format
        '2024-01-20',      # ✅ Good format
        '21/01/24',        # 🔧 Short year
        '2024-01-22'       # ✅ Good format
    ],
    'status': [
        'pending',      # ✅ Good format
        'SHIPPED',      # 🔧 Wrong case
        'Processing',   # 🔧 Wrong case
        'delivered',    # ✅ Good format
        'Cancelled',    # 🔧 Wrong case
        'pending',      # ✅ Good format
        'shipped',      # ✅ Good format
        'PROCESSING'    # 🔧 Wrong case
    ],
    'phone_number': [
        '+1-555-123-4567',    # ✅ Good format
        '555.234.5678',       # 🔧 Dot separator
        '(555) 345-6789',     # 🔧 Parentheses
        '555-456-7890',       # 🔧 Missing country code
        '+1 555 567 8901',    # 🔧 Space separator
        '5556789012',         # 🔧 No separators
        '+1-555-789-0123',    # ✅ Good format
        '555 890 1234'        # 🔧 Space separator
    ]
}

# Create DataFrame
df_messy = pd.DataFrame(messy_orders)

print(f"📊 Created messy dataset with {len(df_messy)} orders")
print(f"📋 Columns: {list(df_messy.columns)}")
print(f"\n🔍 Sample Messy Data:")
display(df_messy)

# Show data types
print(f"\n📈 Data Types:")
for col, dtype in df_messy.dtypes.items():
    print(f"  {col}: {dtype}")

# Show data quality issues
print(f"\n🚨 Data Quality Issues Identified:")
print(f"  - Mixed case in text fields")
print(f"  - Inconsistent date formats")
print(f"  - Currency symbols in prices")
print(f"  - String numbers instead of numeric types")
print(f"  - Inconsistent phone number formats")
print(f"  - Extra spaces in text")
print(f"  - Duplicate order IDs")

## 🧽 Data Cleaning - The Foundation

Data cleaning is the first and most important step in transformation. Let's clean our messy data step by step!

In [ ]:
# Data cleaning system
print("🧽 Data Cleaning System")
print("=" * 25)

@dataclass
class CleaningResult:
    """Container for cleaning results"""
    success: bool
    original_records: int
    cleaned_records: int
    cleaning_operations: List[str]
    cleaning_time: float
    data: Optional[pd.DataFrame] = None
    issues_found: List[str] = None
    issues_fixed: List[str] = None

class DataCleaner:
    """Comprehensive data cleaning system"""
    
    def __init__(self):
        self.cleaning_stats = {
            'operations_performed': [],
            'issues_found': [],
            'issues_fixed': [],
            'records_affected': 0
        }
        
        print(f"🧽 Data cleaner initialized")
    
    def clean_dataset(self, df: pd.DataFrame) -> CleaningResult:
        """
        Perform comprehensive data cleaning
        
        Args:
            df (pd.DataFrame): Dataset to clean
        
        Returns:
            CleaningResult: Cleaning results and cleaned data
        """
        start_time = datetime.now()
        original_records = len(df)
        
        print(f"🧽 Starting data cleaning for {original_records} records")
        
        try:
            # Make a copy to avoid modifying original
            cleaned_df = df.copy()
            operations = []
            issues_found = []
            issues_fixed = []
            
            # Step 1: Remove exact duplicates
            before_dedup = len(cleaned_df)
            cleaned_df = cleaned_df.drop_duplicates()
            after_dedup = len(cleaned_df)
            
            if before_dedup != after_dedup:
                duplicates_removed = before_dedup - after_dedup
                operations.append(f"Removed {duplicates_removed} exact duplicate records")
                issues_found.append(f"Found {duplicates_removed} exact duplicates")
                issues_fixed.append(f"Removed {duplicates_removed} exact duplicates")
            
            # Step 2: Clean text fields
            text_cleaning_result = self._clean_text_fields(cleaned_df)
            cleaned_df = text_cleaning_result['data']
            operations.extend(text_cleaning_result['operations'])
            issues_found.extend(text_cleaning_result['issues_found'])
            issues_fixed.extend(text_cleaning_result['issues_fixed'])
            
            # Step 3: Clean numeric fields
            numeric_cleaning_result = self._clean_numeric_fields(cleaned_df)
            cleaned_df = numeric_cleaning_result['data']
            operations.extend(numeric_cleaning_result['operations'])
            issues_found.extend(numeric_cleaning_result['issues_found'])
            issues_fixed.extend(numeric_cleaning_result['issues_fixed'])
            
            # Step 4: Clean date fields
            date_cleaning_result = self._clean_date_fields(cleaned_df)
            cleaned_df = date_cleaning_result['data']
            operations.extend(date_cleaning_result['operations'])
            issues_found.extend(date_cleaning_result['issues_found'])
            issues_fixed.extend(date_cleaning_result['issues_fixed'])
            
            # Step 5: Clean phone numbers
            phone_cleaning_result = self._clean_phone_numbers(cleaned_df)
            cleaned_df = phone_cleaning_result['data']
            operations.extend(phone_cleaning_result['operations'])
            issues_found.extend(phone_cleaning_result['issues_found'])
            issues_fixed.extend(phone_cleaning_result['issues_fixed'])
            
            # Step 6: Handle missing values
            missing_handling_result = self._handle_missing_values(cleaned_df)
            cleaned_df = missing_handling_result['data']
            operations.extend(missing_handling_result['operations'])
            issues_found.extend(missing_handling_result['issues_found'])
            issues_fixed.extend(missing_handling_result['issues_fixed'])
            
            # Calculate cleaning time
            cleaning_time = (datetime.now() - start_time).total_seconds()
            
            result = CleaningResult(
                success=True,
                original_records=original_records,
                cleaned_records=len(cleaned_df),
                cleaning_operations=operations,
                cleaning_time=cleaning_time,
                data=cleaned_df,
                issues_found=issues_found,
                issues_fixed=issues_fixed
            )
            
            print(f"✅ Data cleaning completed in {cleaning_time:.3f} seconds")
            print(f"📊 Records: {original_records} → {len(cleaned_df)}")
            print(f"🔧 Operations performed: {len(operations)}")
            print(f"🚨 Issues found: {len(issues_found)}")
            print(f"✅ Issues fixed: {len(issues_fixed)}")
            
            return result
            
        except Exception as e:
            cleaning_time = (datetime.now() - start_time).total_seconds()
            print(f"❌ Data cleaning failed: {str(e)}")
            
            return CleaningResult(
                success=False,
                original_records=original_records,
                cleaned_records=0,
                cleaning_operations=[],
                cleaning_time=cleaning_time,
                data=None,
                issues_found=[f"Cleaning failed: {str(e)}"],
                issues_fixed=[]
            )
    
    def _clean_text_fields(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Clean text fields"""
        operations = []
        issues_found = []
        issues_fixed = []
        
        text_fields = ['customer_name', 'product_name', 'status']
        
        for field in text_fields:
            if field in df.columns:
                original_values = df[field].copy()
                
                # Remove extra spaces
                df[field] = df[field].astype(str).str.strip()
                df[field] = df[field].str.replace(r'\s+', ' ', regex=True)
                
                # Standardize case based on field type
                if field == 'customer_name':
                    # Title case for names
                    df[field] = df[field].str.title()
                    # Fix common title issues
                    df[field] = df[field].str.replace(r"\bDr\b", "Dr.", regex=True)
                    df[field] = df[field].str.replace(r"\bMr\b", "Mr.", regex=True)
                    df[field] = df[field].str.replace(r"\bMs\b", "Ms.", regex=True)
                    df[field] = df[field].str.replace(r"\bMrs\b", "Mrs.", regex=True)
                    
                elif field == 'product_name':
                    # Title case for products
                    df[field] = df[field].str.title()
                    # Fix brand names
                    brand_fixes = {
                        'Iphone': 'iPhone',
                        'Macbook': 'MacBook',
                        'Airpods': 'AirPods',
                        'Ipad': 'iPad'
                    }
                    for wrong, correct in brand_fixes.items():
                        df[field] = df[field].str.replace(wrong, correct)
                        
                elif field == 'status':
                    # Lowercase for status
                    df[field] = df[field].str.lower()
                
                # Count changes
                changes = (original_values != df[field]).sum()
                if changes > 0:
                    operations.append(f"Cleaned {field}: {changes} values standardized")
                    issues_found.append(f"Found {changes} inconsistent {field} values")
                    issues_fixed.append(f"Standardized {changes} {field} values")
        
        # Clean email addresses
        if 'customer_email' in df.columns:
            original_emails = df['customer_email'].copy()
            df['customer_email'] = df['customer_email'].astype(str).str.lower().str.strip()
            
            # Count changes
            email_changes = (original_emails != df['customer_email']).sum()
            if email_changes > 0:
                operations.append(f"Cleaned customer_email: {email_changes} values standardized")
                issues_found.append(f"Found {email_changes} inconsistent email formats")
                issues_fixed.append(f"Standardized {email_changes} email addresses")
        
        return {
            'data': df,
            'operations': operations,
            'issues_found': issues_found,
            'issues_fixed': issues_fixed
        }
    
    def _clean_numeric_fields(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Clean numeric fields"""
        operations = []
        issues_found = []
        issues_fixed = []
        
        # Clean price field
        if 'price' in df.columns:
            original_prices = df['price'].copy()
            
            # Convert to string first
            df['price'] = df['price'].astype(str)
            
            # Remove currency symbols and text
            df['price'] = df['price'].str.replace('$', '', regex=False)
            df['price'] = df['price'].str.replace(' USD', '', regex=False)
            df['price'] = df['price'].str.replace(',', '', regex=False)
            
            # Convert to float
            df['price'] = pd.to_numeric(df['price'], errors='coerce')
            
            # Count changes
            price_changes = (original_prices.astype(str) != df['price'].astype(str)).sum()
            if price_changes > 0:
                operations.append(f"Cleaned price: {price_changes} values converted to numeric")
                issues_found.append(f"Found {price_changes} non-numeric price values")
                issues_fixed.append(f"Converted {price_changes} prices to numeric format")
        
        # Clean quantity field
        if 'quantity' in df.columns:
            original_quantities = df['quantity'].copy()
            
            # Convert to numeric
            df['quantity'] = pd.to_numeric(df['quantity'], errors='coerce')
            df['quantity'] = df['quantity'].astype('Int64')  # Nullable integer
            
            # Count changes
            quantity_changes = (original_quantities.astype(str) != df['quantity'].astype(str)).sum()
            if quantity_changes > 0:
                operations.append(f"Cleaned quantity: {quantity_changes} values converted to integer")
                issues_found.append(f"Found {quantity_changes} non-integer quantity values")
                issues_fixed.append(f"Converted {quantity_changes} quantities to integer format")
        
        return {
            'data': df,
            'operations': operations,
            'issues_found': issues_found,
            'issues_fixed': issues_fixed
        }
    
    def _clean_date_fields(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Clean date fields"""
        operations = []
        issues_found = []
        issues_fixed = []
        
        if 'order_date' in df.columns:
            original_dates = df['order_date'].copy()
            
            # Try to parse dates with multiple formats
            date_formats = [
                '%Y-%m-%d',      # 2024-01-15
                '%m/%d/%Y',      # 01/15/2024
                '%d-%m-%Y',      # 15-01-2024
                '%Y/%m/%d',      # 2024/01/15
                '%d %b %Y',      # 15 Jan 2024
                '%d/%m/%y'       # 15/01/24
            ]
            
            parsed_dates = []
            for date_str in df['order_date']:
                parsed_date = None
                for fmt in date_formats:
                    try:
                        parsed_date = pd.to_datetime(date_str, format=fmt)
                        break
                    except:
                        continue
                
                if parsed_date is None:
                    try:
                        parsed_date = pd.to_datetime(date_str, infer_datetime_format=True)
                    except:
                        parsed_date = pd.NaT
                
                parsed_dates.append(parsed_date)
            
            df['order_date'] = parsed_dates
            
            # Count successful conversions
            successful_conversions = df['order_date'].notna().sum()
            failed_conversions = df['order_date'].isna().sum()
            
            if successful_conversions > 0:
                operations.append(f"Cleaned order_date: {successful_conversions} dates standardized")
                issues_found.append(f"Found mixed date formats in {len(df)} records")
                issues_fixed.append(f"Standardized {successful_conversions} dates to ISO format")
            
            if failed_conversions > 0:
                issues_found.append(f"Found {failed_conversions} unparseable dates")
        
        return {
            'data': df,
            'operations': operations,
            'issues_found': issues_found,
            'issues_fixed': issues_fixed
        }
    
    def _clean_phone_numbers(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Clean phone number fields"""
        operations = []
        issues_found = []
        issues_fixed = []
        
        if 'phone_number' in df.columns:
            original_phones = df['phone_number'].copy()
            
            # Standardize phone numbers to +1-XXX-XXX-XXXX format
            standardized_phones = []
            
            for phone in df['phone_number']:
                if pd.isna(phone):
                    standardized_phones.append(phone)
                    continue
                
                # Extract digits only
                digits = re.sub(r'\D', '', str(phone))
                
                # Handle different formats
                if len(digits) == 10:  # US number without country code
                    formatted = f"+1-{digits[:3]}-{digits[3:6]}-{digits[6:]}"
                elif len(digits) == 11 and digits.startswith('1'):  # US number with country code
                    formatted = f"+1-{digits[1:4]}-{digits[4:7]}-{digits[7:]}"
                else:
                    formatted = phone  # Keep original if can't parse
                
                standardized_phones.append(formatted)
            
            df['phone_number'] = standardized_phones
            
            # Count changes
            phone_changes = (original_phones != df['phone_number']).sum()
            if phone_changes > 0:
                operations.append(f"Cleaned phone_number: {phone_changes} numbers standardized")
                issues_found.append(f"Found {phone_changes} inconsistent phone formats")
                issues_fixed.append(f"Standardized {phone_changes} phone numbers")
        
        return {
            'data': df,
            'operations': operations,
            'issues_found': issues_found,
            'issues_fixed': issues_fixed
        }
    
    def _handle_missing_values(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Handle missing values intelligently"""
        operations = []
        issues_found = []
        issues_fixed = []
        
        # Count missing values
        missing_counts = df.isnull().sum()
        total_missing = missing_counts.sum()
        
        if total_missing > 0:
            issues_found.append(f"Found {total_missing} missing values across all columns")
            
            # Handle missing values by column
            for column, missing_count in missing_counts.items():
                if missing_count > 0:
                    if column in ['customer_email', 'phone_number']:
                        # Keep as null for optional contact info
                        operations.append(f"Kept {missing_count} null values in {column} (optional field)")
                    elif column == 'status':
                        # Fill with default status
                        df[column] = df[column].fillna('pending')
                        operations.append(f"Filled {missing_count} missing {column} values with 'pending'")
                        issues_fixed.append(f"Filled {missing_count} missing status values")
                    elif column == 'quantity':
                        # Fill with 1
                        df[column] = df[column].fillna(1)
                        operations.append(f"Filled {missing_count} missing {column} values with 1")
                        issues_fixed.append(f"Filled {missing_count} missing quantity values")
        
        return {
            'data': df,
            'operations': operations,
            'issues_found': issues_found,
            'issues_fixed': issues_fixed
        }

# Test the data cleaner
cleaner = DataCleaner()
cleaning_result = cleaner.clean_dataset(df_messy)

if cleaning_result.success:
    print(f"\n🎉 Data cleaning successful!")
    print(f"\n📊 Cleaning Summary:")
    print(f"  Original records: {cleaning_result.original_records}")
    print(f"  Cleaned records: {cleaning_result.cleaned_records}")
    print(f"  Cleaning time: {cleaning_result.cleaning_time:.3f} seconds")
    
    print(f"\n🔧 Operations Performed:")
    for i, operation in enumerate(cleaning_result.cleaning_operations, 1):
        print(f"  {i}. {operation}")
    
    print(f"\n✅ Issues Fixed:")
    for i, fix in enumerate(cleaning_result.issues_fixed, 1):
        print(f"  {i}. {fix}")
    
    print(f"\n📋 Cleaned Data Sample:")
    display(cleaning_result.data.head())
    
    # Store cleaned data for next steps
    df_cleaned = cleaning_result.data
    
else:
    print(f"❌ Data cleaning failed!")
    print(f"Issues: {cleaning_result.issues_found}")

## 📏 Data Standardization - Making Everything Consistent

After cleaning, we need to standardize our data to ensure consistency across all records. This includes formatting, naming conventions, and data structures.

In [ ]:
# Data standardization system
print("📏 Data Standardization System")
print("=" * 35)

@dataclass
class StandardizationResult:
    """Container for standardization results"""
    success: bool
    records_processed: int
    fields_standardized: int
    standardization_operations: List[str]
    standardization_time: float
    data: Optional[pd.DataFrame] = None
    before_after_examples: Dict[str, List[Tuple[str, str]]] = None

class DataStandardizer:
    """Comprehensive data standardization system"""
    
    def __init__(self):
        # Define standardization rules
        self.standardization_rules = {
            'order_id': {
                'format': 'ORD-YYYY-NNN',
                'pattern': r'^ORD-\d{4}-\d{3}$'
            },
            'product_categories': {
                'iPhone': 'Smartphones',
                'MacBook': 'Laptops',
                'iPad': 'Tablets',
                'AirPods': 'Audio',
                'Apple Watch': 'Wearables',
                'Nintendo Switch': 'Gaming',
                'Samsung Galaxy': 'Smartphones',
                'Sony': 'Audio'
            },
            'status_mapping': {
                'pending': 'pending',
                'processing': 'processing', 
                'shipped': 'shipped',
                'delivered': 'delivered',
                'cancelled': 'cancelled'
            }
        }
        
        print(f"📏 Data standardizer initialized")
        print(f"  Standardization rules loaded: {len(self.standardization_rules)}")
    
    def standardize_dataset(self, df: pd.DataFrame) -> StandardizationResult:
        """
        Perform comprehensive data standardization
        
        Args:
            df (pd.DataFrame): Dataset to standardize
        
        Returns:
            StandardizationResult: Standardization results
        """
        start_time = datetime.now()
        records_processed = len(df)
        
        print(f"📏 Starting data standardization for {records_processed} records")
        
        try:
            # Make a copy
            standardized_df = df.copy()
            operations = []
            fields_standardized = 0
            before_after_examples = {}
            
            # Step 1: Standardize order IDs
            if 'order_id' in standardized_df.columns:
                order_id_result = self._standardize_order_ids(standardized_df)
                standardized_df = order_id_result['data']
                operations.extend(order_id_result['operations'])
                before_after_examples['order_id'] = order_id_result['examples']
                if order_id_result['standardized']:
                    fields_standardized += 1
            
            # Step 2: Add product categories
            if 'product_name' in standardized_df.columns:
                category_result = self._add_product_categories(standardized_df)
                standardized_df = category_result['data']
                operations.extend(category_result['operations'])
                before_after_examples['product_category'] = category_result['examples']
                if category_result['added']:
                    fields_standardized += 1
            
            # Step 3: Standardize customer information
            customer_result = self._standardize_customer_info(standardized_df)
            standardized_df = customer_result['data']
            operations.extend(customer_result['operations'])
            before_after_examples.update(customer_result['examples'])
            fields_standardized += customer_result['fields_standardized']
            
            # Step 4: Add calculated fields
            calculated_result = self._add_calculated_fields(standardized_df)
            standardized_df = calculated_result['data']
            operations.extend(calculated_result['operations'])
            fields_standardized += calculated_result['fields_added']
            
            # Step 5: Standardize column names
            column_result = self._standardize_column_names(standardized_df)
            standardized_df = column_result['data']
            operations.extend(column_result['operations'])
            
            # Calculate standardization time
            standardization_time = (datetime.now() - start_time).total_seconds()
            
            result = StandardizationResult(
                success=True,
                records_processed=records_processed,
                fields_standardized=fields_standardized,
                standardization_operations=operations,
                standardization_time=standardization_time,
                data=standardized_df,
                before_after_examples=before_after_examples
            )
            
            print(f"✅ Data standardization completed in {standardization_time:.3f} seconds")
            print(f"📊 Records processed: {records_processed}")
            print(f"📏 Fields standardized: {fields_standardized}")
            print(f"🔧 Operations performed: {len(operations)}")
            
            return result
            
        except Exception as e:
            standardization_time = (datetime.now() - start_time).total_seconds()
            print(f"❌ Data standardization failed: {str(e)}")
            
            return StandardizationResult(
                success=False,
                records_processed=records_processed,
                fields_standardized=0,
                standardization_operations=[],
                standardization_time=standardization_time,
                data=None
            )
    
    def _standardize_order_ids(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Standardize order ID format"""
        operations = []
        examples = []
        standardized = False
        
        if 'order_id' in df.columns:
            original_ids = df['order_id'].copy()
            
            # Standardize to ORD-YYYY-NNN format
            standardized_ids = []
            
            for order_id in df['order_id']:
                if pd.isna(order_id):
                    standardized_ids.append(order_id)
                    continue
                
                order_str = str(order_id).upper()
                
                # Handle different formats
                if order_str.startswith('ORD-') and len(order_str.split('-')) == 3:
                    # Already in correct format
                    standardized_ids.append(order_str)
                elif order_str.startswith('ORD_'):
                    # Replace underscores with dashes
                    standardized_ids.append(order_str.replace('_', '-'))
                elif order_str.startswith('ORDER-'):
                    # Shorten ORDER to ORD
                    standardized_ids.append(order_str.replace('ORDER-', 'ORD-'))
                elif re.match(r'^\d{4}-\d{3}$', order_str):
                    # Add ORD prefix
                    standardized_ids.append(f'ORD-{order_str}')
                elif re.match(r'^ORD-\d{2}-\d{3}$', order_str):
                    # Fix short year
                    parts = order_str.split('-')
                    year = f'20{parts[1]}' if int(parts[1]) < 50 else f'19{parts[1]}'
                    standardized_ids.append(f'ORD-{year}-{parts[2]}')
                else:
                    # Keep original if can't parse
                    standardized_ids.append(order_str)
            
            df['order_id'] = standardized_ids
            
            # Count changes and create examples
            changes = (original_ids != df['order_id']).sum()
            if changes > 0:
                standardized = True
                operations.append(f"Standardized {changes} order IDs to ORD-YYYY-NNN format")
                
                # Create before/after examples
                for i, (old, new) in enumerate(zip(original_ids, df['order_id'])):
                    if old != new and len(examples) < 3:
                        examples.append((str(old), str(new)))
        
        return {
            'data': df,
            'operations': operations,
            'examples': examples,
            'standardized': standardized
        }
    
    def _add_product_categories(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add product categories based on product names"""
        operations = []
        examples = []
        added = False
        
        if 'product_name' in df.columns:
            # Create product category column
            categories = []
            
            for product in df['product_name']:
                if pd.isna(product):
                    categories.append('Unknown')
                    continue
                
                product_str = str(product)
                category = 'Other'  # Default category
                
                # Match against category rules
                for key, cat in self.standardization_rules['product_categories'].items():
                    if key.lower() in product_str.lower():
                        category = cat
                        break
                
                categories.append(category)
            
            df['product_category'] = categories
            added = True
            
            operations.append(f"Added product_category field with {len(set(categories))} unique categories")
            
            # Create examples
            for i, (product, category) in enumerate(zip(df['product_name'], categories)):
                if len(examples) < 3 and category != 'Other':
                    examples.append((str(product), str(category)))
        
        return {
            'data': df,
            'operations': operations,
            'examples': examples,
            'added': added
        }
    
    def _standardize_customer_info(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Standardize customer information"""
        operations = []
        examples = {}
        fields_standardized = 0
        
        # Extract customer domain from email
        if 'customer_email' in df.columns:
            domains = []
            for email in df['customer_email']:
                if pd.isna(email) or '@' not in str(email):
                    domains.append('Unknown')
                else:
                    domain = str(email).split('@')[1]
                    domains.append(domain)
            
            df['customer_domain'] = domains
            operations.append(f"Added customer_domain field extracted from email addresses")
            examples['customer_domain'] = [(df['customer_email'].iloc[0], domains[0])] if len(domains) > 0 else []
            fields_standardized += 1
        
        # Create customer type based on email domain
        if 'customer_domain' in df.columns:
            customer_types = []
            for domain in df['customer_domain']:
                if domain == 'Unknown':
                    customer_types.append('Unknown')
                elif domain in ['gmail.com', 'yahoo.com', 'hotmail.com', 'outlook.com']:
                    customer_types.append('Personal')
                else:
                    customer_types.append('Business')
            
            df['customer_type'] = customer_types
            operations.append(f"Added customer_type field based on email domain")
            fields_standardized += 1
        
        return {
            'data': df,
            'operations': operations,
            'examples': examples,
            'fields_standardized': fields_standardized
        }
    
    def _add_calculated_fields(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add calculated fields"""
        operations = []
        fields_added = 0
        
        # Calculate total amount
        if 'price' in df.columns and 'quantity' in df.columns:
            df['total_amount'] = df['price'] * df['quantity']
            operations.append("Added total_amount field (price × quantity)")
            fields_added += 1
        
        # Add order value category
        if 'total_amount' in df.columns:
            value_categories = []
            for amount in df['total_amount']:
                if pd.isna(amount):
                    value_categories.append('Unknown')
                elif amount < 100:
                    value_categories.append('Low')
                elif amount < 500:
                    value_categories.append('Medium')
                elif amount < 1000:
                    value_categories.append('High')
                else:
                    value_categories.append('Premium')
            
            df['order_value_category'] = value_categories
            operations.append("Added order_value_category field based on total amount")
            fields_added += 1
        
        # Add processing timestamp
        df['processed_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        operations.append("Added processed_at timestamp")
        fields_added += 1
        
        # Add record hash for tracking
        record_hashes = []
        for _, row in df.iterrows():
            # Create hash from key fields
            key_fields = ['order_id', 'customer_email', 'product_name']
            hash_string = ''.join([str(row.get(field, '')) for field in key_fields])
            record_hash = hashlib.md5(hash_string.encode()).hexdigest()[:8]
            record_hashes.append(record_hash)
        
        df['record_hash'] = record_hashes
        operations.append("Added record_hash for data lineage tracking")
        fields_added += 1
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _standardize_column_names(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Standardize column names to snake_case"""
        operations = []
        
        # Convert column names to snake_case
        original_columns = df.columns.tolist()
        standardized_columns = []
        
        for col in original_columns:
            # Convert to snake_case
            snake_case = re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower()
            snake_case = re.sub(r'[^a-z0-9_]', '_', snake_case)
            snake_case = re.sub(r'_+', '_', snake_case).strip('_')
            standardized_columns.append(snake_case)
        
        # Rename columns
        column_mapping = dict(zip(original_columns, standardized_columns))
        df = df.rename(columns=column_mapping)
        
        # Count changes
        changes = sum(1 for old, new in column_mapping.items() if old != new)
        if changes > 0:
            operations.append(f"Standardized {changes} column names to snake_case")
        
        return {
            'data': df,
            'operations': operations
        }

# Test the data standardizer
standardizer = DataStandardizer()
standardization_result = standardizer.standardize_dataset(df_cleaned)

if standardization_result.success:
    print(f"\n🎉 Data standardization successful!")
    
    print(f"\n🔧 Standardization Operations:")
    for i, operation in enumerate(standardization_result.standardization_operations, 1):
        print(f"  {i}. {operation}")
    
    print(f"\n📋 Before/After Examples:")
    for field, examples in standardization_result.before_after_examples.items():
        if examples:
            print(f"  {field}:")
            for before, after in examples[:2]:  # Show first 2 examples
                print(f"    '{before}' → '{after}'")
    
    print(f"\n📊 Standardized Data Sample:")
    display(standardization_result.data.head())
    
    print(f"\n📏 New Columns Added:")
    new_columns = set(standardization_result.data.columns) - set(df_cleaned.columns)
    for col in sorted(new_columns):
        print(f"  - {col}")
    
    # Store standardized data for next steps
    df_standardized = standardization_result.data
    
else:
    print(f"❌ Data standardization failed!")

## ➕ Data Enrichment - Adding Value and Intelligence

Data enrichment adds valuable information and insights to your dataset. This includes calculated metrics, business intelligence, and external data integration.

In [ ]:
# Data enrichment system
print("➕ Data Enrichment System")
print("=" * 30)

@dataclass
class EnrichmentResult:
    """Container for enrichment results"""
    success: bool
    records_processed: int
    fields_added: int
    enrichment_operations: List[str]
    enrichment_time: float
    data: Optional[pd.DataFrame] = None
    enrichment_summary: Dict[str, Any] = None

class DataEnricher:
    """Comprehensive data enrichment system"""
    
    def __init__(self):
        # Define enrichment rules and lookup data
        self.product_data = {
            'iPhone 15': {'brand': 'Apple', 'release_year': 2023, 'avg_rating': 4.5},
            'MacBook Pro': {'brand': 'Apple', 'release_year': 2023, 'avg_rating': 4.7},
            'iPad Air': {'brand': 'Apple', 'release_year': 2022, 'avg_rating': 4.4},
            'AirPods Pro': {'brand': 'Apple', 'release_year': 2022, 'avg_rating': 4.6},
            'Apple Watch Series 9': {'brand': 'Apple', 'release_year': 2023, 'avg_rating': 4.3},
            'Nintendo Switch Oled': {'brand': 'Nintendo', 'release_year': 2021, 'avg_rating': 4.8},
            'Samsung Galaxy S24': {'brand': 'Samsung', 'release_year': 2024, 'avg_rating': 4.4},
            'Sony Wh-1000Xm5': {'brand': 'Sony', 'release_year': 2022, 'avg_rating': 4.7}
        }
        
        self.seasonal_factors = {
            1: 0.8,   # January - post-holiday low
            2: 0.9,   # February
            3: 1.0,   # March
            4: 1.0,   # April
            5: 1.1,   # May
            6: 1.0,   # June
            7: 1.0,   # July
            8: 1.1,   # August - back to school
            9: 1.2,   # September - back to school
            10: 1.1,  # October
            11: 1.4,  # November - Black Friday
            12: 1.5   # December - Holiday season
        }
        
        print(f"➕ Data enricher initialized")
        print(f"  Product database: {len(self.product_data)} products")
        print(f"  Seasonal factors: {len(self.seasonal_factors)} months")
    
    def enrich_dataset(self, df: pd.DataFrame) -> EnrichmentResult:
        """
        Perform comprehensive data enrichment
        
        Args:
            df (pd.DataFrame): Dataset to enrich
        
        Returns:
            EnrichmentResult: Enrichment results
        """
        start_time = datetime.now()
        records_processed = len(df)
        
        print(f"➕ Starting data enrichment for {records_processed} records")
        
        try:
            # Make a copy
            enriched_df = df.copy()
            operations = []
            fields_added = 0
            
            # Step 1: Add product information
            product_result = self._add_product_information(enriched_df)
            enriched_df = product_result['data']
            operations.extend(product_result['operations'])
            fields_added += product_result['fields_added']
            
            # Step 2: Add customer insights
            customer_result = self._add_customer_insights(enriched_df)
            enriched_df = customer_result['data']
            operations.extend(customer_result['operations'])
            fields_added += customer_result['fields_added']
            
            # Step 3: Add temporal features
            temporal_result = self._add_temporal_features(enriched_df)
            enriched_df = temporal_result['data']
            operations.extend(temporal_result['operations'])
            fields_added += temporal_result['fields_added']
            
            # Step 4: Add business metrics
            business_result = self._add_business_metrics(enriched_df)
            enriched_df = business_result['data']
            operations.extend(business_result['operations'])
            fields_added += business_result['fields_added']
            
            # Step 5: Add quality scores
            quality_result = self._add_quality_scores(enriched_df)
            enriched_df = quality_result['data']
            operations.extend(quality_result['operations'])
            fields_added += quality_result['fields_added']
            
            # Create enrichment summary
            enrichment_summary = self._create_enrichment_summary(enriched_df)
            
            # Calculate enrichment time
            enrichment_time = (datetime.now() - start_time).total_seconds()
            
            result = EnrichmentResult(
                success=True,
                records_processed=records_processed,
                fields_added=fields_added,
                enrichment_operations=operations,
                enrichment_time=enrichment_time,
                data=enriched_df,
                enrichment_summary=enrichment_summary
            )
            
            print(f"✅ Data enrichment completed in {enrichment_time:.3f} seconds")
            print(f"📊 Records processed: {records_processed}")
            print(f"➕ Fields added: {fields_added}")
            print(f"🔧 Operations performed: {len(operations)}")
            
            return result
            
        except Exception as e:
            enrichment_time = (datetime.now() - start_time).total_seconds()
            print(f"❌ Data enrichment failed: {str(e)}")
            
            return EnrichmentResult(
                success=False,
                records_processed=records_processed,
                fields_added=0,
                enrichment_operations=[],
                enrichment_time=enrichment_time,
                data=None
            )
    
    def _add_product_information(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add product information from lookup data"""
        operations = []
        fields_added = 0
        
        if 'product_name' in df.columns:
            # Add brand information
            brands = []
            release_years = []
            ratings = []
            
            for product in df['product_name']:
                if pd.isna(product):
                    brands.append('Unknown')
                    release_years.append(None)
                    ratings.append(None)
                    continue
                
                product_str = str(product)
                product_info = self.product_data.get(product_str, {})
                
                brands.append(product_info.get('brand', 'Unknown'))
                release_years.append(product_info.get('release_year', None))
                ratings.append(product_info.get('avg_rating', None))
            
            df['product_brand'] = brands
            df['product_release_year'] = release_years
            df['product_avg_rating'] = ratings
            
            operations.append("Added product_brand, product_release_year, and product_avg_rating")
            fields_added += 3
            
            # Add product age
            current_year = datetime.now().year
            product_ages = []
            for year in release_years:
                if year is None:
                    product_ages.append(None)
                else:
                    product_ages.append(current_year - year)
            
            df['product_age_years'] = product_ages
            operations.append("Added product_age_years calculated from release year")
            fields_added += 1
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _add_customer_insights(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add customer insights and segmentation"""
        operations = []
        fields_added = 0
        
        # Customer segmentation based on order value
        if 'total_amount' in df.columns:
            customer_segments = []
            for amount in df['total_amount']:
                if pd.isna(amount):
                    customer_segments.append('Unknown')
                elif amount < 200:
                    customer_segments.append('Budget')
                elif amount < 500:
                    customer_segments.append('Standard')
                elif amount < 1000:
                    customer_segments.append('Premium')
                else:
                    customer_segments.append('VIP')
            
            df['customer_segment'] = customer_segments
            operations.append("Added customer_segment based on order value")
            fields_added += 1
        
        # Customer name analysis
        if 'customer_name' in df.columns:
            name_lengths = []
            has_title = []
            
            for name in df['customer_name']:
                if pd.isna(name):
                    name_lengths.append(None)
                    has_title.append(False)
                else:
                    name_str = str(name)
                    name_lengths.append(len(name_str))
                    has_title.append(any(title in name_str for title in ['Dr.', 'Mr.', 'Ms.', 'Mrs.']))
            
            df['customer_name_length'] = name_lengths
            df['customer_has_title'] = has_title
            operations.append("Added customer_name_length and customer_has_title")
            fields_added += 2
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _add_temporal_features(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add temporal features and seasonal analysis"""
        operations = []
        fields_added = 0
        
        if 'order_date' in df.columns:
            # Extract date components
            df['order_year'] = df['order_date'].dt.year
            df['order_month'] = df['order_date'].dt.month
            df['order_day'] = df['order_date'].dt.day
            df['order_weekday'] = df['order_date'].dt.day_name()
            df['order_quarter'] = df['order_date'].dt.quarter
            
            operations.append("Added order_year, order_month, order_day, order_weekday, order_quarter")
            fields_added += 5
            
            # Add seasonal factors
            seasonal_factors = []
            for month in df['order_month']:
                if pd.isna(month):
                    seasonal_factors.append(1.0)
                else:
                    seasonal_factors.append(self.seasonal_factors.get(int(month), 1.0))
            
            df['seasonal_factor'] = seasonal_factors
            operations.append("Added seasonal_factor based on order month")
            fields_added += 1
            
            # Add days since order
            current_date = datetime.now()
            days_since_order = []
            for order_date in df['order_date']:
                if pd.isna(order_date):
                    days_since_order.append(None)
                else:
                    delta = current_date - order_date.to_pydatetime()
                    days_since_order.append(delta.days)
            
            df['days_since_order'] = days_since_order
            operations.append("Added days_since_order calculated from current date")
            fields_added += 1
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _add_business_metrics(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add business intelligence metrics"""
        operations = []
        fields_added = 0
        
        # Calculate profit margin (simplified)
        if 'total_amount' in df.columns:
            # Assume 30% profit margin
            df['estimated_profit'] = df['total_amount'] * 0.30
            operations.append("Added estimated_profit (30% of total_amount)")
            fields_added += 1
        
        # Add order priority based on multiple factors
        if all(col in df.columns for col in ['total_amount', 'customer_segment', 'product_avg_rating']):
            priorities = []
            for _, row in df.iterrows():
                priority_score = 0
                
                # Amount factor
                if not pd.isna(row['total_amount']):
                    if row['total_amount'] > 1000:
                        priority_score += 3
                    elif row['total_amount'] > 500:
                        priority_score += 2
                    else:
                        priority_score += 1
                
                # Customer segment factor
                segment_scores = {'VIP': 3, 'Premium': 2, 'Standard': 1, 'Budget': 0}
                priority_score += segment_scores.get(row.get('customer_segment', 'Budget'), 0)
                
                # Product rating factor
                if not pd.isna(row.get('product_avg_rating')):
                    if row['product_avg_rating'] >= 4.5:
                        priority_score += 2
                    elif row['product_avg_rating'] >= 4.0:
                        priority_score += 1
                
                # Convert to priority level
                if priority_score >= 7:
                    priorities.append('Critical')
                elif priority_score >= 5:
                    priorities.append('High')
                elif priority_score >= 3:
                    priorities.append('Medium')
                else:
                    priorities.append('Low')
            
            df['order_priority'] = priorities
            operations.append("Added order_priority based on amount, segment, and rating")
            fields_added += 1
        
        # Add risk score
        risk_scores = []
        for _, row in df.iterrows():
            risk_score = 0
            
            # High value orders have higher risk
            if not pd.isna(row.get('total_amount')) and row['total_amount'] > 1000:
                risk_score += 2
            
            # New products have higher risk
            if not pd.isna(row.get('product_age_years')) and row['product_age_years'] < 1:
                risk_score += 1
            
            # Business customers have lower risk
            if row.get('customer_type') == 'Business':
                risk_score -= 1
            
            # Convert to risk category
            if risk_score >= 3:
                risk_scores.append('High')
            elif risk_score >= 1:
                risk_scores.append('Medium')
            else:
                risk_scores.append('Low')
        
        df['risk_score'] = risk_scores
        operations.append("Added risk_score based on multiple risk factors")
        fields_added += 1
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _add_quality_scores(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Add data quality scores for each record"""
        operations = []
        fields_added = 0
        
        # Calculate completeness score for each record
        completeness_scores = []
        required_fields = ['order_id', 'customer_name', 'product_name', 'quantity', 'price']
        
        for _, row in df.iterrows():
            filled_fields = 0
            for field in required_fields:
                if field in row.index and pd.notna(row[field]) and str(row[field]).strip() != '':
                    filled_fields += 1
            
            completeness_score = (filled_fields / len(required_fields)) * 100
            completeness_scores.append(completeness_score)
        
        df['data_completeness_score'] = completeness_scores
        operations.append("Added data_completeness_score for each record")
        fields_added += 1
        
        # Add overall data quality grade
        quality_grades = []
        for score in completeness_scores:
            if score >= 95:
                quality_grades.append('A')
            elif score >= 85:
                quality_grades.append('B')
            elif score >= 70:
                quality_grades.append('C')
            elif score >= 50:
                quality_grades.append('D')
            else:
                quality_grades.append('F')
        
        df['data_quality_grade'] = quality_grades
        operations.append("Added data_quality_grade based on completeness score")
        fields_added += 1
        
        return {
            'data': df,
            'operations': operations,
            'fields_added': fields_added
        }
    
    def _create_enrichment_summary(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Create summary of enrichment results"""
        summary = {
            'total_records': len(df),
            'total_columns': len(df.columns),
            'customer_segments': df['customer_segment'].value_counts().to_dict() if 'customer_segment' in df.columns else {},
            'product_brands': df['product_brand'].value_counts().to_dict() if 'product_brand' in df.columns else {},
            'order_priorities': df['order_priority'].value_counts().to_dict() if 'order_priority' in df.columns else {},
            'risk_distribution': df['risk_score'].value_counts().to_dict() if 'risk_score' in df.columns else {},
            'quality_grades': df['data_quality_grade'].value_counts().to_dict() if 'data_quality_grade' in df.columns else {},
            'avg_completeness_score': df['data_completeness_score'].mean() if 'data_completeness_score' in df.columns else 0
        }
        
        return summary

# Test the data enricher
enricher = DataEnricher()
enrichment_result = enricher.enrich_dataset(df_standardized)

if enrichment_result.success:
    print(f"\n🎉 Data enrichment successful!")
    
    print(f"\n🔧 Enrichment Operations:")
    for i, operation in enumerate(enrichment_result.enrichment_operations, 1):
        print(f"  {i}. {operation}")
    
    print(f"\n📊 Enrichment Summary:")
    summary = enrichment_result.enrichment_summary
    print(f"  Total records: {summary['total_records']}")
    print(f"  Total columns: {summary['total_columns']}")
    print(f"  Average completeness score: {summary['avg_completeness_score']:.1f}%")
    
    print(f"\n👥 Customer Segments:")
    for segment, count in summary['customer_segments'].items():
        print(f"  {segment}: {count}")
    
    print(f"\n🏷️ Product Brands:")
    for brand, count in summary['product_brands'].items():
        print(f"  {brand}: {count}")
    
    print(f"\n⚡ Order Priorities:")
    for priority, count in summary['order_priorities'].items():
        print(f"  {priority}: {count}")
    
    print(f"\n📋 Enriched Data Sample:")
    # Show key enriched columns
    key_columns = ['order_id', 'customer_name', 'product_name', 'total_amount', 
                   'customer_segment', 'product_brand', 'order_priority', 'data_quality_grade']
    available_columns = [col for col in key_columns if col in enrichment_result.data.columns]
    display(enrichment_result.data[available_columns].head())
    
    # Store enriched data for final analysis
    df_enriched = enrichment_result.data
    
else:
    print(f"❌ Data enrichment failed!")

## 🔄 Complete Transformation Pipeline

Let's put everything together into a complete, reusable transformation pipeline that you can use in production!

In [ ]:
# Complete transformation pipeline
print("🔄 Complete Data Transformation Pipeline")
print("=" * 45)

@dataclass
class TransformationResult:
    """Container for complete transformation results"""
    success: bool
    pipeline_name: str
    original_records: int
    final_records: int
    original_columns: int
    final_columns: int
    total_time: float
    cleaning_result: Optional[CleaningResult] = None
    standardization_result: Optional[StandardizationResult] = None
    enrichment_result: Optional[EnrichmentResult] = None
    final_data: Optional[pd.DataFrame] = None
    transformation_summary: Dict[str, Any] = None

class DataTransformationPipeline:
    """Complete data transformation pipeline"""
    
    def __init__(self, pipeline_name: str = "data_transformation_pipeline"):
        self.pipeline_name = pipeline_name
        
        # Initialize components
        self.cleaner = DataCleaner()
        self.standardizer = DataStandardizer()
        self.enricher = DataEnricher()
        
        # Pipeline configuration
        self.enable_cleaning = True
        self.enable_standardization = True
        self.enable_enrichment = True
        
        print(f"🔄 Transformation pipeline '{pipeline_name}' initialized")
        print(f"  Components: Cleaning, Standardization, Enrichment")
    
    def transform_dataset(self, df: pd.DataFrame, 
                         enable_cleaning: bool = True,
                         enable_standardization: bool = True,
                         enable_enrichment: bool = True) -> TransformationResult:
        """
        Execute complete transformation pipeline
        
        Args:
            df (pd.DataFrame): Dataset to transform
            enable_cleaning (bool): Enable cleaning stage
            enable_standardization (bool): Enable standardization stage
            enable_enrichment (bool): Enable enrichment stage
        
        Returns:
            TransformationResult: Complete transformation results
        """
        start_time = datetime.now()
        original_records = len(df)
        original_columns = len(df.columns)
        
        print(f"🚀 Starting transformation pipeline: {self.pipeline_name}")
        print(f"📊 Input dataset: {original_records} records, {original_columns} columns")
        print(f"🔧 Stages enabled: Cleaning={enable_cleaning}, Standardization={enable_standardization}, Enrichment={enable_enrichment}")
        
        try:
            current_data = df.copy()
            cleaning_result = None
            standardization_result = None
            enrichment_result = None
            
            # Stage 1: Data Cleaning
            if enable_cleaning:
                print(f"\n🧽 Stage 1: Data Cleaning")
                print(f"  Input: {len(current_data)} records, {len(current_data.columns)} columns")
                
                cleaning_result = self.cleaner.clean_dataset(current_data)
                
                if cleaning_result.success:
                    current_data = cleaning_result.data
                    print(f"  ✅ Cleaning completed: {len(current_data)} records, {len(current_data.columns)} columns")
                    print(f"  🔧 Operations: {len(cleaning_result.cleaning_operations)}")
                    print(f"  ✅ Issues fixed: {len(cleaning_result.issues_fixed)}")
                else:
                    raise Exception(f"Data cleaning failed: {cleaning_result.issues_found}")
            else:
                print(f"\n⏭️ Stage 1: Data Cleaning (Skipped)")
            
            # Stage 2: Data Standardization
            if enable_standardization:
                print(f"\n📏 Stage 2: Data Standardization")
                print(f"  Input: {len(current_data)} records, {len(current_data.columns)} columns")
                
                standardization_result = self.standardizer.standardize_dataset(current_data)
                
                if standardization_result.success:
                    current_data = standardization_result.data
                    print(f"  ✅ Standardization completed: {len(current_data)} records, {len(current_data.columns)} columns")
                    print(f"  📏 Fields standardized: {standardization_result.fields_standardized}")
                else:
                    raise Exception("Data standardization failed")
            else:
                print(f"\n⏭️ Stage 2: Data Standardization (Skipped)")
            
            # Stage 3: Data Enrichment
            if enable_enrichment:
                print(f"\n➕ Stage 3: Data Enrichment")
                print(f"  Input: {len(current_data)} records, {len(current_data.columns)} columns")
                
                enrichment_result = self.enricher.enrich_dataset(current_data)
                
                if enrichment_result.success:
                    current_data = enrichment_result.data
                    print(f"  ✅ Enrichment completed: {len(current_data)} records, {len(current_data.columns)} columns")
                    print(f"  ➕ Fields added: {enrichment_result.fields_added}")
                else:
                    raise Exception("Data enrichment failed")
            else:
                print(f"\n⏭️ Stage 3: Data Enrichment (Skipped)")
            
            # Calculate final metrics
            total_time = (datetime.now() - start_time).total_seconds()
            final_records = len(current_data)
            final_columns = len(current_data.columns)
            
            # Create transformation summary
            transformation_summary = self._create_transformation_summary(
                df, current_data, cleaning_result, standardization_result, enrichment_result
            )
            
            result = TransformationResult(
                success=True,
                pipeline_name=self.pipeline_name,
                original_records=original_records,
                final_records=final_records,
                original_columns=original_columns,
                final_columns=final_columns,
                total_time=total_time,
                cleaning_result=cleaning_result,
                standardization_result=standardization_result,
                enrichment_result=enrichment_result,
                final_data=current_data,
                transformation_summary=transformation_summary
            )
            
            print(f"\n🎉 Transformation pipeline completed successfully!")
            print(f"⏱️ Total time: {total_time:.3f} seconds")
            print(f"📊 Final dataset: {final_records} records, {final_columns} columns")
            print(f"📈 Records retained: {(final_records/original_records)*100:.1f}%")
            print(f"➕ Columns added: {final_columns - original_columns}")
            
            return result
            
        except Exception as e:
            total_time = (datetime.now() - start_time).total_seconds()
            print(f"\n❌ Transformation pipeline failed: {str(e)}")
            
            return TransformationResult(
                success=False,
                pipeline_name=self.pipeline_name,
                original_records=original_records,
                final_records=0,
                original_columns=original_columns,
                final_columns=0,
                total_time=total_time,
                final_data=None
            )
    
    def _create_transformation_summary(self, original_df: pd.DataFrame, 
                                     final_df: pd.DataFrame,
                                     cleaning_result: Optional[CleaningResult],
                                     standardization_result: Optional[StandardizationResult],
                                     enrichment_result: Optional[EnrichmentResult]) -> Dict[str, Any]:
        """Create comprehensive transformation summary"""
        
        summary = {
            'pipeline_name': self.pipeline_name,
            'transformation_timestamp': datetime.now().isoformat(),
            'data_flow': {
                'original_records': len(original_df),
                'final_records': len(final_df),
                'records_retained_pct': (len(final_df) / len(original_df)) * 100,
                'original_columns': len(original_df.columns),
                'final_columns': len(final_df.columns),
                'columns_added': len(final_df.columns) - len(original_df.columns)
            },
            'stage_results': {}
        }
        
        # Add stage-specific results
        if cleaning_result:
            summary['stage_results']['cleaning'] = {
                'success': cleaning_result.success,
                'operations_count': len(cleaning_result.cleaning_operations),
                'issues_fixed_count': len(cleaning_result.issues_fixed),
                'time_seconds': cleaning_result.cleaning_time
            }
        
        if standardization_result:
            summary['stage_results']['standardization'] = {
                'success': standardization_result.success,
                'fields_standardized': standardization_result.fields_standardized,
                'operations_count': len(standardization_result.standardization_operations),
                'time_seconds': standardization_result.standardization_time
            }
        
        if enrichment_result:
            summary['stage_results']['enrichment'] = {
                'success': enrichment_result.success,
                'fields_added': enrichment_result.fields_added,
                'operations_count': len(enrichment_result.enrichment_operations),
                'time_seconds': enrichment_result.enrichment_time,
                'avg_completeness_score': enrichment_result.enrichment_summary.get('avg_completeness_score', 0)
            }
        
        # Add data quality metrics
        if 'data_quality_grade' in final_df.columns:
            summary['data_quality'] = {
                'quality_grades': final_df['data_quality_grade'].value_counts().to_dict(),
                'avg_completeness': final_df['data_completeness_score'].mean() if 'data_completeness_score' in final_df.columns else 0
            }
        
        return summary
    
    def generate_transformation_report(self, result: TransformationResult) -> str:
        """Generate comprehensive transformation report"""
        report = []
        report.append("# 🔄 Data Transformation Pipeline Report")
        report.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append("=" * 60)
        report.append("")
        
        # Executive Summary
        report.append("## 🎯 Executive Summary")
        report.append("")
        status = "✅ SUCCESS" if result.success else "❌ FAILED"
        report.append(f"**Pipeline Status:** {status}")
        report.append(f"**Pipeline Name:** {result.pipeline_name}")
        report.append(f"**Total Execution Time:** {result.total_time:.3f} seconds")
        report.append(f"**Records Processed:** {result.original_records:,} → {result.final_records:,}")
        report.append(f"**Columns:** {result.original_columns} → {result.final_columns} (+{result.final_columns - result.original_columns})")
        report.append(f"**Data Retention Rate:** {(result.final_records/result.original_records)*100:.1f}%")
        report.append("")
        
        # Stage Results
        report.append("## 🔧 Stage Results")
        report.append("")
        
        if result.cleaning_result:
            cleaning = result.cleaning_result
            status_icon = "✅" if cleaning.success else "❌"
            report.append(f"### {status_icon} Data Cleaning")
            report.append(f"- **Execution Time:** {cleaning.cleaning_time:.3f} seconds")
            report.append(f"- **Operations Performed:** {len(cleaning.cleaning_operations)}")
            report.append(f"- **Issues Fixed:** {len(cleaning.issues_fixed)}")
            report.append(f"- **Records Retained:** {cleaning.cleaned_records}/{cleaning.original_records}")
            report.append("")
        
        if result.standardization_result:
            standardization = result.standardization_result
            status_icon = "✅" if standardization.success else "❌"
            report.append(f"### {status_icon} Data Standardization")
            report.append(f"- **Execution Time:** {standardization.standardization_time:.3f} seconds")
            report.append(f"- **Fields Standardized:** {standardization.fields_standardized}")
            report.append(f"- **Operations Performed:** {len(standardization.standardization_operations)}")
            report.append("")
        
        if result.enrichment_result:
            enrichment = result.enrichment_result
            status_icon = "✅" if enrichment.success else "❌"
            report.append(f"### {status_icon} Data Enrichment")
            report.append(f"- **Execution Time:** {enrichment.enrichment_time:.3f} seconds")
            report.append(f"- **Fields Added:** {enrichment.fields_added}")
            report.append(f"- **Operations Performed:** {len(enrichment.enrichment_operations)}")
            if enrichment.enrichment_summary:
                avg_completeness = enrichment.enrichment_summary.get('avg_completeness_score', 0)
                report.append(f"- **Average Data Completeness:** {avg_completeness:.1f}%")
            report.append("")
        
        # Data Quality Summary
        if result.final_data is not None and 'data_quality_grade' in result.final_data.columns:
            report.append("## 📊 Data Quality Summary")
            report.append("")
            quality_grades = result.final_data['data_quality_grade'].value_counts()
            for grade, count in quality_grades.items():
                percentage = (count / len(result.final_data)) * 100
                report.append(f"- **Grade {grade}:** {count} records ({percentage:.1f}%)")
            report.append("")
        
        # Recommendations
        report.append("## 💡 Recommendations")
        report.append("")
        if result.success:
            report.append("- ✅ **Transformation completed successfully**")
            report.append("- 📊 **Monitor data quality trends over time**")
            report.append("- 🔄 **Consider automating this pipeline for regular execution**")
            report.append("- 📈 **Use enriched data for advanced analytics and ML**")
        else:
            report.append("- ❌ **Review and fix transformation errors**")
            report.append("- 🔍 **Check input data quality and format**")
            report.append("- 🛠️ **Update transformation rules as needed**")
        
        report.append("")
        report.append("---")
        report.append(f"*Report generated by Data Transformation Pipeline v1.0*")
        
        return "\n".join(report)

# Test the complete transformation pipeline
print("\n🧪 Testing Complete Transformation Pipeline")
print("=" * 45)

# Initialize pipeline
pipeline = DataTransformationPipeline("E-commerce Order Transformation")

# Run complete transformation
transformation_result = pipeline.transform_dataset(
    df_messy,
    enable_cleaning=True,
    enable_standardization=True,
    enable_enrichment=True
)

if transformation_result.success:
    print(f"\n📊 Transformation Summary:")
    summary = transformation_result.transformation_summary
    
    print(f"  Pipeline: {summary['pipeline_name']}")
    print(f"  Records: {summary['data_flow']['original_records']} → {summary['data_flow']['final_records']}")
    print(f"  Columns: {summary['data_flow']['original_columns']} → {summary['data_flow']['final_columns']}")
    print(f"  Retention Rate: {summary['data_flow']['records_retained_pct']:.1f}%")
    
    print(f"\n🔧 Stage Performance:")
    for stage, results in summary['stage_results'].items():
        status = "✅" if results['success'] else "❌"
        print(f"  {status} {stage.title()}: {results['time_seconds']:.3f}s")
    
    # Show final transformed data
    print(f"\n📋 Final Transformed Data (Sample):")
    key_columns = ['order_id', 'customer_name', 'product_name', 'total_amount', 
                   'customer_segment', 'product_brand', 'order_priority', 'data_quality_grade']
    available_columns = [col for col in key_columns if col in transformation_result.final_data.columns]
    display(transformation_result.final_data[available_columns].head())
    
    # Generate and display report
    report = pipeline.generate_transformation_report(transformation_result)
    print(f"\n📋 Transformation Report Generated ({len(report):,} characters)")
    
    # Show report preview
    print(f"\n" + "=" * 60)
    print(report[:1000] + "..." if len(report) > 1000 else report)
    print("=" * 60)
    
else:
    print(f"❌ Transformation pipeline failed!")

## 📊 Before vs After Analysis

Let's create a comprehensive comparison between our original messy data and the final transformed data to see the impact of our transformation pipeline!

In [ ]:
# Before vs After analysis
print("📊 Before vs After Transformation Analysis")
print("=" * 45)

def analyze_transformation_impact(original_df: pd.DataFrame, 
                                transformed_df: pd.DataFrame) -> Dict[str, Any]:
    """
    Analyze the impact of data transformation
    
    Args:
        original_df (pd.DataFrame): Original messy data
        transformed_df (pd.DataFrame): Transformed clean data
    
    Returns:
        Dict[str, Any]: Analysis results
    """
    analysis = {
        'data_structure': {},
        'data_quality': {},
        'data_richness': {},
        'business_value': {}
    }
    
    # Data Structure Analysis
    analysis['data_structure'] = {
        'records': {
            'before': len(original_df),
            'after': len(transformed_df),
            'change': len(transformed_df) - len(original_df),
            'retention_rate': (len(transformed_df) / len(original_df)) * 100
        },
        'columns': {
            'before': len(original_df.columns),
            'after': len(transformed_df.columns),
            'added': len(transformed_df.columns) - len(original_df.columns),
            'new_columns': list(set(transformed_df.columns) - set(original_df.columns))
        }
    }
    
    # Data Quality Analysis
    original_missing = original_df.isnull().sum().sum()
    transformed_missing = transformed_df.isnull().sum().sum()
    
    analysis['data_quality'] = {
        'missing_values': {
            'before': original_missing,
            'after': transformed_missing,
            'improvement': original_missing - transformed_missing
        },
        'completeness': {
            'before': ((original_df.size - original_missing) / original_df.size) * 100,
            'after': ((transformed_df.size - transformed_missing) / transformed_df.size) * 100
        }
    }
    
    # Add quality grades if available
    if 'data_quality_grade' in transformed_df.columns:
        quality_distribution = transformed_df['data_quality_grade'].value_counts().to_dict()
        analysis['data_quality']['quality_grades'] = quality_distribution
    
    # Data Richness Analysis
    analysis['data_richness'] = {
        'calculated_fields': [],
        'enrichment_fields': [],
        'standardized_fields': []
    }
    
    # Identify different types of new fields
    new_columns = set(transformed_df.columns) - set(original_df.columns)
    
    for col in new_columns:
        if 'total' in col.lower() or 'amount' in col.lower():
            analysis['data_richness']['calculated_fields'].append(col)
        elif any(word in col.lower() for word in ['brand', 'category', 'segment', 'priority', 'risk']):
            analysis['data_richness']['enrichment_fields'].append(col)
        else:
            analysis['data_richness']['standardized_fields'].append(col)
    
    # Business Value Analysis
    analysis['business_value'] = {
        'analytics_ready': True,
        'ml_ready': 'data_quality_grade' in transformed_df.columns,
        'business_insights': [],
        'automation_potential': 'High'
    }
    
    # Identify business insights
    if 'customer_segment' in transformed_df.columns:
        analysis['business_value']['business_insights'].append('Customer Segmentation Available')
    
    if 'product_brand' in transformed_df.columns:
        analysis['business_value']['business_insights'].append('Brand Analysis Possible')
    
    if 'order_priority' in transformed_df.columns:
        analysis['business_value']['business_insights'].append('Order Prioritization Enabled')
    
    if 'seasonal_factor' in transformed_df.columns:
        analysis['business_value']['business_insights'].append('Seasonal Analysis Available')
    
    return analysis

# Perform the analysis
if transformation_result.success:
    impact_analysis = analyze_transformation_impact(df_messy, transformation_result.final_data)
    
    print("📊 TRANSFORMATION IMPACT ANALYSIS")
    print("=" * 40)
    
    # Data Structure Impact
    print("\n🏗️ Data Structure Impact:")
    structure = impact_analysis['data_structure']
    print(f"  Records: {structure['records']['before']} → {structure['records']['after']} ({structure['records']['retention_rate']:.1f}% retained)")
    print(f"  Columns: {structure['columns']['before']} → {structure['columns']['after']} (+{structure['columns']['added']} new)")
    
    # Data Quality Impact
    print("\n✨ Data Quality Impact:")
    quality = impact_analysis['data_quality']
    print(f"  Missing Values: {quality['missing_values']['before']} → {quality['missing_values']['after']} ({quality['missing_values']['improvement']} improvement)")
    print(f"  Completeness: {quality['completeness']['before']:.1f}% → {quality['completeness']['after']:.1f}%")
    
    if 'quality_grades' in quality:
        print(f"  Quality Grades:")
        for grade, count in quality['quality_grades'].items():
            percentage = (count / len(transformation_result.final_data)) * 100
            print(f"    Grade {grade}: {count} records ({percentage:.1f}%)")
    
    # Data Richness Impact
    print("\n🎯 Data Enrichment Impact:")
    richness = impact_analysis['data_richness']
    print(f"  Calculated Fields: {len(richness['calculated_fields'])} ({', '.join(richness['calculated_fields'][:3])})")
    print(f"  Enrichment Fields: {len(richness['enrichment_fields'])} ({', '.join(richness['enrichment_fields'][:3])})")
    print(f"  Standardized Fields: {len(richness['standardized_fields'])} ({', '.join(richness['standardized_fields'][:3])})")
    
    # Business Value Impact
    print("\n💼 Business Value Impact:")
    business = impact_analysis['business_value']
    print(f"  Analytics Ready: {'✅ Yes' if business['analytics_ready'] else '❌ No'}")
    print(f"  ML Ready: {'✅ Yes' if business['ml_ready'] else '❌ No'}")
    print(f"  Automation Potential: {business['automation_potential']}")
    
    print(f"\n🎯 Business Insights Enabled:")
    for insight in business['business_insights']:
        print(f"    ✅ {insight}")
    
    # Create visualizations
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Data Structure Comparison
    categories = ['Records', 'Columns']
    before_values = [structure['records']['before'], structure['columns']['before']]
    after_values = [structure['records']['after'], structure['columns']['after']]
    
    x = np.arange(len(categories))
    width = 0.35
    
    ax1.bar(x - width/2, before_values, width, label='Before', alpha=0.7, color='red')
    ax1.bar(x + width/2, after_values, width, label='After', alpha=0.7, color='green')
    ax1.set_xlabel('Data Structure')
    ax1.set_ylabel('Count')
    ax1.set_title('Data Structure: Before vs After')
    ax1.set_xticks(x)
    ax1.set_xticklabels(categories)
    ax1.legend()
    
    # Add value labels on bars
    for i, (before, after) in enumerate(zip(before_values, after_values)):
        ax1.text(i - width/2, before + 0.1, str(before), ha='center', va='bottom')
        ax1.text(i + width/2, after + 0.1, str(after), ha='center', va='bottom')
    
    # 2. Data Quality Grades (if available)
    if 'quality_grades' in quality:
        grades = list(quality['quality_grades'].keys())
        counts = list(quality['quality_grades'].values())
        colors = ['green', 'lightgreen', 'yellow', 'orange', 'red'][:len(grades)]
        
        ax2.pie(counts, labels=grades, autopct='%1.1f%%', colors=colors)
        ax2.set_title('Data Quality Grade Distribution')
    else:
        ax2.text(0.5, 0.5, 'Quality Grades\nNot Available', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Data Quality Grades')
    
    # 3. Customer Segment Distribution (if available)
    if 'customer_segment' in transformation_result.final_data.columns:
        segment_counts = transformation_result.final_data['customer_segment'].value_counts()
        ax3.bar(segment_counts.index, segment_counts.values, alpha=0.7)
        ax3.set_xlabel('Customer Segment')
        ax3.set_ylabel('Count')
        ax3.set_title('Customer Segmentation')
        ax3.tick_params(axis='x', rotation=45)
    else:
        ax3.text(0.5, 0.5, 'Customer Segmentation\nNot Available', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Customer Segmentation')
    
    # 4. Order Priority Distribution (if available)
    if 'order_priority' in transformation_result.final_data.columns:
        priority_counts = transformation_result.final_data['order_priority'].value_counts()
        colors = {'Critical': 'red', 'High': 'orange', 'Medium': 'yellow', 'Low': 'green'}
        bar_colors = [colors.get(priority, 'blue') for priority in priority_counts.index]
        
        ax4.bar(priority_counts.index, priority_counts.values, color=bar_colors, alpha=0.7)
        ax4.set_xlabel('Order Priority')
        ax4.set_ylabel('Count')
        ax4.set_title('Order Priority Distribution')
    else:
        ax4.text(0.5, 0.5, 'Order Priority\nNot Available', ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Order Priority Distribution')
    
    plt.tight_layout()
    plt.show()
    
    # Show side-by-side comparison
    print(f"\n📋 Side-by-Side Data Comparison:")
    print(f"\n🔴 BEFORE (Original Messy Data):")
    display(df_messy.head(3))
    
    print(f"\n🟢 AFTER (Transformed Clean Data):")
    key_columns = ['order_id', 'customer_name', 'product_name', 'total_amount', 
                   'customer_segment', 'product_brand', 'order_priority', 'data_quality_grade']
    available_columns = [col for col in key_columns if col in transformation_result.final_data.columns]
    display(transformation_result.final_data[available_columns].head(3))
    
else:
    print("❌ Cannot perform analysis - transformation failed")

## 🎯 Key Takeaways

Congratulations! You've completed the data transformation tutorial. Here's what you've mastered:

### ✅ **Core Transformation Skills**
- **🧽 Data Cleaning**: Removing inconsistencies, fixing formats, handling missing values
- **📏 Data Standardization**: Creating consistent formats and structures
- **➕ Data Enrichment**: Adding calculated fields, business intelligence, and metadata
- **🔄 Pipeline Orchestration**: Building complete transformation workflows
- **📊 Impact Analysis**: Measuring transformation effectiveness

### ✅ **Advanced Techniques Mastered**
- **Text Standardization**: Case normalization, brand name fixing, format consistency
- **Numeric Cleaning**: Currency symbol removal, type conversion, validation
- **Date Standardization**: Multi-format parsing, ISO standardization
- **Phone Number Formatting**: Pattern recognition and standardization
- **Business Intelligence**: Customer segmentation, product categorization, risk scoring
- **Quality Scoring**: Record-level quality assessment and grading

### ✅ **Production-Ready Features**
- **Comprehensive Error Handling**: Graceful failure management at every stage
- **Performance Monitoring**: Execution time tracking and optimization
- **Detailed Reporting**: Before/after analysis and transformation reports
- **Configurable Pipelines**: Enable/disable stages based on requirements
- **Data Lineage**: Hash-based tracking for data provenance

### ✅ **Business Value Created**
- **Analytics-Ready Data**: Clean, consistent data for analysis
- **ML-Ready Features**: Quality scores and enriched attributes
- **Business Intelligence**: Customer segments, product insights, risk assessment
- **Operational Efficiency**: Automated data preparation and quality assurance

---

## 🚀 What's Next?

In the next tutorial, **"06_building_complete_pipeline.ipynb"**, you'll learn:
- 🔗 How to connect all pipeline components together
- ⚡ Building end-to-end automated workflows
- 📊 Advanced monitoring and alerting systems
- 🔄 Error recovery and retry mechanisms
- 🎯 Production deployment strategies

### 🎯 **Practice Exercise**

Before moving to the next tutorial, try this exercise:

1. **Create your own messy dataset** for a different domain (healthcare, finance, retail)
2. **Identify transformation needs** specific to that domain
3. **Customize the transformation pipeline** with domain-specific rules
4. **Add custom enrichment logic** relevant to your business case
5. **Measure the transformation impact** using our analysis framework
6. **Create a transformation report** for stakeholders

### 💡 **Advanced Transformation Ideas:**
- **Fuzzy Matching**: Handle similar but not identical values
- **External Data Integration**: Enrich with third-party data sources
- **Machine Learning Enhancement**: Use ML for data quality prediction
- **Real-time Transformation**: Stream processing for live data
- **A/B Testing**: Compare different transformation strategies

---

**Outstanding work mastering data transformation! 🎉**

You now have the skills to transform any messy dataset into clean, enriched, analytics-ready data. Data transformation is where raw data becomes valuable business intelligence - you're creating the foundation for all downstream analytics and machine learning.

**Happy Transforming! 🔄**